In [1]:
%pylab inline
import os
import concurrent.futures
import json
from keras.applications import inception_v3, InceptionV3, VGG19, InceptionResNetV2
from keras.applications import imagenet_utils
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import decode_predictions
from PIL import Image
import h5py as h5py
import ssl
import pytesseract
ssl._create_default_https_context = ssl._create_unverified_context


Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.


In [2]:
class ImageFeatureExtractor:
    def __init__(self, model='inceptionv3', resize_shape=(299, 299)):
        Models = {"vgg19": VGG19, "inceptionv3": InceptionV3, "inceptionresnetv2": InceptionResNetV2}
        self.model = Models[model.lower()](include_top=False, weights='imagenet', pooling='avg')
        self.resize_shape = resize_shape
        if model in ['inceptionv3']:
            self.preprocess_input = inception_v3.preprocess_input
        else:
            self.preprocess_input = imagenet_utils.preprocess_input

    def preprocess(self, image_file):
        # load an image in PIL format
        original = load_img(image_file, target_size=self.resize_shape) #299, 299
        numpy_image = img_to_array(original)
        image_batch = np.expand_dims(numpy_image, axis=0)
        '''
        print('image batch size', image_batch.shape)
        plt.imshow(np.uint8(image_batch[0]))
        '''  
        return self.preprocess_input(image_batch.copy())
    
    def extract_features(self, image):
        features = self.model.predict(image)
        #features = decode_predictions(prediction)
        return features[0]
    
    

In [3]:
%%time

extractor0 = ImageFeatureExtractor(model='inceptionv3', resize_shape = (299, 299))
# extractor1 = ImageFeatureExtractor(model='vgg19', resize_shape = (224, 224))
# extractor2 = ImageFeatureExtractor(model='inceptionresnetv2', resize_shape = (299, 299))

CPU times: user 14.9 s, sys: 624 ms, total: 15.6 s
Wall time: 15.9 s


In [5]:
image_file = 'Frame_00001118.jpg'

# img = extractor0.preprocess(image_file)
# labels = extractor0.extract_labels(img, top=3)
# print(labels)

img = extractor0.preprocess(image_file)
features = extractor0.extract_features(img)
print(features.shape)
#np.save('Frame_00012403', features)
#feature_np = np.load('Frame_00012403.npy')
#print(feature_np.shape)
# img = extractor2.preprocess(image_file)
# labels = extractor2.extract_labels(img, top=3)
# print(labels)

(2048,)


In [ ]:
%%time
json_file = open('./Movies/Life Is Strange 1/output 1/Life Is Strange.json')
database = json.loads(json_file.read())
for block in database:
    folder = block['screenshots']['image_folder']
    sessions = block['screenshots']['image_info']
    for session in sessions:
        for key, image in session['frames'].items():
            img_name = image['image_file']
            file_path = os.path.join(folder, img_name)
            img = extractor0.preprocess(file_path)
            features = extractor0.extract_features(img)
            #save to file
            np_filename = img_name[:img_name.rfind('.')]
            np_dir = folder + '_feature'
            if not os.path.exists(np_dir):
                os.makedirs(np_dir)
                
            np_file = os.path.join(np_dir, np_filename)
            np.save(np_file, features)
